In [ ]:
!nvidia-smi

Tue Jan 26 08:35:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd '/content/drive/MyDrive/vin/NLP/nmt_attention2'

/content/drive/MyDrive/vin/NLP/nmt_attention2


In [ ]:
!pip install tensorflow-addons==0.11.2

     |████████████████████████████████| 1.1MB 10.9MB/s 
  Found existing installation: tensorflow-addons 0.8.3
    Uninstalling tensorflow-addons-0.8.3:
      Successfully uninstalled tensorflow-addons-0.8.3


In [ ]:
import logging 
import tensorflow as tf
import tensorflow_addons as tfa

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time
import pickle
import gc

/usr/local/lib/python3.6/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:68: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.4.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.4.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [ ]:
log_file = open('logs/log_bahdanau.log', 'a+')

In [ ]:
# load tokenizer
with open('tokenizer/tokenizer.pickle', 'rb') as f:
  data = pickle.load(f)
  en_tokenizer = data['en_tokenizer']
  vi_tokenizer = data['vi_tokenizer']

In [ ]:
def preprocess_sentence(s):
  s = s.lower()
  s = s.strip()
  s = '<s> ' + s + ' </s>'
  return s

en_data_tensor_path = 'sequences_data/en_data.pickle'
vi_data_tensor_path = 'sequences_data/vi_data.pickle'

with open(en_data_tensor_path, 'rb') as f:
  en_data = pickle.load(f)
  en_train = en_data['en_train_tensor']
  en_dev = en_data['en_dev_tensor']
  en_test = en_data['en_test_tensor']

with open(vi_data_tensor_path, 'rb') as f:
  vi_data = pickle.load(f)
  vi_train = vi_data['vi_train_tensor']
  vi_dev = vi_data['vi_dev_tensor']
  vi_test = vi_data['vi_test_tensor']

max_length_en, max_length_vi = en_train.shape[1], vi_train.shape[1]

# save information
with open('information/infor_bahdanau.pickle', 'wb') as handle:
    pickle.dump(
        {'max_length_en': max_length_en, 
         'max_length_vi': max_length_vi,
         'en_tokenizer': en_tokenizer,
         'vi_tokenizer': vi_tokenizer,
         'attention': 'bahdanau',
         'en_example': en_train[0],
         'vi_example': vi_train[0]
        }, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
BUFFER_SIZE = 32000
BATCH_SIZE = 128
steps_per_epoch = len(en_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_en_size = len(en_tokenizer.word_index)+1
vocab_vi_size = len(vi_tokenizer.word_index)+1

train_dataset = tf.data.Dataset.from_tensor_slices((en_train, vi_train)).shuffle(BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, name="embedding")

    self.lstm_layer = tf.keras.layers.LSTM(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform',
                                   name="lstm")
    
  def call(self, x, hidden):
    x = self.embedding(x)
    output, h, c = self.lstm_layer(x, initial_state = hidden)
    return output, h, c

  def initialize_hidden_state(self):
    return [tf.zeros((self.batch_sz, self.enc_units)), tf.zeros((self.batch_sz, self.enc_units))]

In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz, attention_type='luong'):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units

    self.attention_type = attention_type

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, embeddings_initializer='uniform')

    self.fc = tf.keras.layers.Dense(vocab_size)

    self.decoder_rnn_cell = tf.keras.layers.LSTMCell(self.dec_units)

    self.sampler = tfa.seq2seq.sampler.TrainingSampler()

    self.attention_mechanism = self.build_attention_mechanism(self.dec_units, 
                                                              None, self.batch_sz*[max_length_en], self.attention_type)

    self.rnn_cell = self.build_rnn_cell(batch_sz)

    self.decoder = tfa.seq2seq.BasicDecoder(self.rnn_cell, sampler=self.sampler, output_layer=self.fc)

  def build_rnn_cell(self, batch_sz):
    rnn_cell = tfa.seq2seq.AttentionWrapper(self.decoder_rnn_cell,
                                  self.attention_mechanism, attention_layer_size=self.dec_units)
    return rnn_cell

  def build_attention_mechanism(self, dec_units, memory, memory_sequence_length, attention_type='luong'):

    if(attention_type=='bahdanau'):
      return tfa.seq2seq.BahdanauAttention(units=dec_units, memory=memory, memory_sequence_length=memory_sequence_length)
    else:
      return tfa.seq2seq.LuongAttention(units=dec_units, memory=memory, memory_sequence_length=memory_sequence_length)

  def build_initial_state(self, batch_sz, encoder_state, Dtype):
    decoder_initial_state = self.rnn_cell.get_initial_state(batch_size=batch_sz, dtype=Dtype)
    decoder_initial_state = decoder_initial_state.clone(cell_state=encoder_state)
    return decoder_initial_state

  def call(self, inputs, initial_state):
    x = self.embedding(inputs)
    outputs, _, _ = self.decoder(x, initial_state=initial_state, sequence_length=self.batch_sz*[max_length_vi-1])
    return outputs

In [ ]:
optimizer = tf.keras.optimizers.Adam()

def loss_function(real, pred):
  cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
  loss = cross_entropy(y_true=real, y_pred=pred)
  mask = tf.logical_not(tf.math.equal(real,0))
  mask = tf.cast(mask, dtype=loss.dtype)
  loss = mask* loss
  loss = tf.reduce_mean(loss)
  return loss

In [ ]:
encoder = Encoder(vocab_en_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_vi_size, embedding_dim, units, BATCH_SIZE, 'bahdanau')

checkpoint_dir = './checkpoints/bahdanau_cp'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
# BasicDecoder
def evaluate_sentence(sentence):
  sentence = preprocess_sentence(sentence)

  inputs = [en_tokenizer.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                          maxlen=max_length_en,
                                                          padding='post')
  inputs = tf.convert_to_tensor(inputs)
  inference_batch_size = inputs.shape[0]
  result = ''

  enc_start_state = [tf.zeros((inference_batch_size, units)), tf.zeros((inference_batch_size,units))]
  enc_out, enc_h, enc_c = encoder(inputs, enc_start_state)

  dec_h = enc_h
  dec_c = enc_c

  start_tokens = tf.fill([inference_batch_size], vi_tokenizer.word_index['<s>'])
  end_token = vi_tokenizer.word_index['</s>']

  greedy_sampler = tfa.seq2seq.GreedyEmbeddingSampler()

  decoder_instance = tfa.seq2seq.BasicDecoder(cell=decoder.rnn_cell, sampler=greedy_sampler, output_layer=decoder.fc)
  decoder.attention_mechanism.setup_memory(enc_out)

  decoder_initial_state = decoder.build_initial_state(inference_batch_size, [enc_h, enc_c], tf.float32)

  decoder_embedding_matrix = decoder.embedding.variables[0]

  outputs, _, _ = decoder_instance(decoder_embedding_matrix, start_tokens = start_tokens, end_token= end_token, initial_state=decoder_initial_state)
  return outputs.sample_id.numpy()

def translate(sentence):
  result = evaluate_sentence(sentence)
  result = vi_tokenizer.sequences_to_texts(result)
  return result

In [ ]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_h, enc_c = encoder(inp, enc_hidden)


    dec_input = targ[ : , :-1 ]
    real = targ[ : , 1: ]

    decoder.attention_mechanism.setup_memory(enc_output)

    decoder_initial_state = decoder.build_initial_state(BATCH_SIZE, [enc_h, enc_c], tf.float32)
    pred = decoder(dec_input, decoder_initial_state)
    logits = pred.rnn_output
    loss = loss_function(real, logits)

  variables = encoder.trainable_variables + decoder.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))

  return loss

EPOCHS = 50
for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(train_dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      log = 'Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy())
      print(log)
      log_file.writelines(log+"\n")

  checkpoint.save(file_prefix = checkpoint_prefix)
  
  log = 'Epoch {} Loss {:.4f} '.format(epoch + 1,
                                      total_loss / steps_per_epoch)
  print(log)
  log_file.writelines(log+'\n')
  log = 'Time taken for 1 epoch {} sec\n'.format(time.time() - start)
  print(log)
  log_file.writelines(log+'\n')

Epoch 1 Batch 0 Loss 0.1901
Epoch 1 Batch 100 Loss 0.2163
Epoch 1 Batch 200 Loss 0.1810
Epoch 1 Batch 300 Loss 0.1926
Epoch 1 Batch 400 Loss 0.1756
Epoch 1 Batch 500 Loss 0.1875
Epoch 1 Batch 600 Loss 0.2033
Epoch 1 Batch 700 Loss 0.1907
Epoch 1 Batch 800 Loss 0.1911
Epoch 1 Batch 900 Loss 0.2180
Epoch 1 Loss 0.1981 
Time taken for 1 epoch 1020.9594633579254 sec

Epoch 2 Batch 0 Loss 0.1827
Epoch 2 Batch 100 Loss 0.1729
Epoch 2 Batch 200 Loss 0.1935
Epoch 2 Batch 300 Loss 0.1808
Epoch 2 Batch 400 Loss 0.1764
Epoch 2 Batch 500 Loss 0.1839
Epoch 2 Batch 600 Loss 0.1911
Epoch 2 Batch 700 Loss 0.1951
Epoch 2 Batch 800 Loss 0.1875
Epoch 2 Batch 900 Loss 0.2045
Epoch 2 Loss 0.1892 
Time taken for 1 epoch 989.3292467594147 sec

Epoch 3 Batch 0 Loss 0.1672
Epoch 3 Batch 100 Loss 0.1920
Epoch 3 Batch 200 Loss 0.1766
Epoch 3 Batch 300 Loss 0.1900
Epoch 3 Batch 400 Loss 0.1824
Epoch 3 Batch 500 Loss 0.1689
Epoch 3 Batch 600 Loss 0.1549
Epoch 3 Batch 700 Loss 0.2094
Epoch 3 Batch 800 Loss 0.1868
E

In [ ]:
log_file.close()

In [ ]:
# Epoch 1 Batch 0 Loss 4.1542
# Epoch 1 Batch 100 Loss 2.6751
# Epoch 1 Batch 200 Loss 2.2813
# Epoch 1 Batch 300 Loss 2.2257
# Epoch 1 Batch 400 Loss 1.9947
# Epoch 1 Batch 500 Loss 2.1493
# Epoch 1 Batch 600 Loss 1.8154
# Epoch 1 Batch 700 Loss 2.0720
# Epoch 1 Batch 800 Loss 1.7620
# Epoch 1 Batch 900 Loss 1.6200
# Epoch 1 Loss 2.0875
# Time taken for 1 epoch 669.3499715328217 sec

# Epoch 2 Batch 0 Loss 1.4498
# Epoch 2 Batch 100 Loss 1.4541
# Epoch 2 Batch 200 Loss 1.3935
# Epoch 2 Batch 300 Loss 1.4523
# Epoch 2 Batch 400 Loss 1.3322
# Epoch 2 Batch 500 Loss 1.2438
# Epoch 2 Batch 600 Loss 1.2254
# Epoch 2 Batch 700 Loss 1.2793
# Epoch 2 Batch 800 Loss 1.1715
# Epoch 2 Batch 900 Loss 1.1007
# Epoch 2 Loss 1.3176
# Time taken for 1 epoch 652.2943453788757 sec

# Epoch 3 Batch 0 Loss 1.1778
# Epoch 3 Batch 100 Loss 1.0570
# Epoch 3 Batch 200 Loss 1.0299
# Epoch 3 Batch 300 Loss 1.1160
# Epoch 3 Batch 400 Loss 1.0518
# Epoch 3 Batch 500 Loss 1.0607
# Epoch 3 Batch 600 Loss 0.9822
# Epoch 3 Batch 700 Loss 1.1073
# Epoch 3 Batch 800 Loss 1.0119
# Epoch 3 Batch 900 Loss 1.0381
# Epoch 3 Loss 1.0407
# Time taken for 1 epoch 649.4882378578186 sec

# Epoch 4 Batch 0 Loss 0.9701
# Epoch 4 Batch 100 Loss 0.9343
# Epoch 4 Batch 200 Loss 0.9873
# Epoch 4 Batch 300 Loss 0.8650
# Epoch 4 Batch 400 Loss 0.9645
# Epoch 4 Batch 500 Loss 0.8636
# Epoch 4 Batch 600 Loss 0.8378
# Epoch 4 Batch 700 Loss 0.9464
# Epoch 4 Batch 800 Loss 0.9650
# Epoch 4 Batch 900 Loss 0.8431
# Epoch 4 Loss 0.8990
# Time taken for 1 epoch 651.2808754444122 sec

# Epoch 5 Batch 0 Loss 0.8022
# Epoch 5 Batch 100 Loss 0.8614
# Epoch 5 Batch 200 Loss 0.7625
# Epoch 5 Batch 300 Loss 0.8247
# Epoch 5 Batch 400 Loss 0.7678
# Epoch 5 Batch 500 Loss 0.8732
# Epoch 5 Batch 600 Loss 0.7645
# Epoch 5 Batch 700 Loss 0.7530
# Epoch 5 Batch 800 Loss 0.8380
# Epoch 5 Batch 900 Loss 0.8369
# Epoch 5 Loss 0.8032
# Time taken for 1 epoch 650.3041110038757 sec

# Epoch 6 Batch 0 Loss 0.7799
# Epoch 6 Batch 100 Loss 0.7404
# Epoch 6 Batch 200 Loss 0.7077
# Epoch 6 Batch 300 Loss 0.7561
# Epoch 6 Batch 400 Loss 0.7539
# Epoch 6 Batch 500 Loss 0.7822
# Epoch 6 Batch 600 Loss 0.7440
# Epoch 6 Batch 700 Loss 0.7387
# Epoch 6 Batch 800 Loss 0.7043
# Epoch 6 Batch 900 Loss 0.7556
# Epoch 6 Loss 0.7335
# Time taken for 1 epoch 649.7862718105316 sec

# Epoch 7 Batch 0 Loss 0.6781
# Epoch 7 Batch 100 Loss 0.6947
# Epoch 7 Batch 200 Loss 0.6841
# Epoch 7 Batch 300 Loss 0.5924
# Epoch 7 Batch 400 Loss 0.6727
# Epoch 7 Batch 500 Loss 0.6376
# Epoch 7 Batch 600 Loss 0.6613
# Epoch 7 Batch 700 Loss 0.7771
# Epoch 7 Batch 800 Loss 0.6250
# Epoch 7 Batch 900 Loss 0.6642
# Epoch 7 Loss 0.6727
# Time taken for 1 epoch 649.1937673091888 sec

# Epoch 8 Batch 0 Loss 0.6667
# Epoch 8 Batch 100 Loss 0.5860
# Epoch 8 Batch 200 Loss 0.6719
# Epoch 8 Batch 300 Loss 0.5738
# Epoch 8 Batch 400 Loss 0.6069
# Epoch 8 Batch 500 Loss 0.5105
# Epoch 8 Batch 600 Loss 0.6737
# Epoch 8 Batch 700 Loss 0.5844
# Epoch 8 Batch 800 Loss 0.6643
# Epoch 8 Batch 900 Loss 0.6837
# Epoch 8 Loss 0.6205
# Time taken for 1 epoch 649.8456366062164 sec

# Epoch 9 Batch 0 Loss 0.6434
# Epoch 9 Batch 100 Loss 0.5676
# Epoch 9 Batch 200 Loss 0.5875
# Epoch 9 Batch 300 Loss 0.5495
# Epoch 9 Batch 400 Loss 0.5069
# Epoch 9 Batch 500 Loss 0.5474
# Epoch 9 Batch 600 Loss 0.5379
# Epoch 9 Batch 700 Loss 0.5484
# Epoch 9 Batch 800 Loss 0.6733
# Epoch 9 Batch 900 Loss 0.7078
# Epoch 9 Loss 0.5708
# Time taken for 1 epoch 651.3016917705536 sec

# Epoch 10 Batch 0 Loss 0.5394
# Epoch 10 Batch 100 Loss 0.5009
# Epoch 10 Batch 200 Loss 0.5534
# Epoch 10 Batch 300 Loss 0.5201
# Epoch 10 Batch 400 Loss 0.5188
# Epoch 10 Batch 500 Loss 0.5799
# Epoch 10 Batch 600 Loss 0.5131
# Epoch 10 Batch 700 Loss 0.4880
# Epoch 10 Batch 800 Loss 0.5090
# Epoch 10 Batch 900 Loss 0.5415
# Epoch 10 Loss 0.5258
# Time taken for 1 epoch 652.1481170654297 sec

# Epoch 11 Batch 0 Loss 0.4501
# Epoch 11 Batch 100 Loss 0.4481
# Epoch 11 Batch 200 Loss 0.4564
# Epoch 11 Batch 300 Loss 0.5208
# Epoch 11 Batch 400 Loss 0.4513
# Epoch 11 Batch 500 Loss 0.4383
# Epoch 11 Batch 600 Loss 0.4938
# Epoch 11 Batch 700 Loss 0.5278
# Epoch 11 Batch 800 Loss 0.5305
# Epoch 11 Batch 900 Loss 0.5004

In [ ]:
# BasicDecoder
def evaluate_sentence(sentence):
  sentence = preprocess_sentence(sentence)

  inputs = [en_tokenizer.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                          maxlen=max_length_en,
                                                          padding='post')
  inputs = tf.convert_to_tensor(inputs)
  inference_batch_size = inputs.shape[0]
  result = ''

  enc_start_state = [tf.zeros((inference_batch_size, units)), tf.zeros((inference_batch_size,units))]
  enc_out, enc_h, enc_c = encoder(inputs, enc_start_state)

  dec_h = enc_h
  dec_c = enc_c

  start_tokens = tf.fill([inference_batch_size], vi_tokenizer.word_index['<start>'])
  end_token = vi_tokenizer.word_index['<end>']

  greedy_sampler = tfa.seq2seq.GreedyEmbeddingSampler()

  decoder_instance = tfa.seq2seq.BasicDecoder(cell=decoder.rnn_cell, sampler=greedy_sampler, output_layer=decoder.fc)
  decoder.attention_mechanism.setup_memory(enc_out)

  decoder_initial_state = decoder.build_initial_state(inference_batch_size, [enc_h, enc_c], tf.float32)

  decoder_embedding_matrix = decoder.embedding.variables[0]

  outputs, _, _ = decoder_instance(decoder_embedding_matrix, start_tokens = start_tokens, end_token= end_token, initial_state=decoder_initial_state)
  return outputs.sample_id.numpy()

def translate(sentence):
  result = evaluate_sentence(sentence)
  print(result)
  result = vi_tokenizer.sequences_to_texts(result)
  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))
  return result

translate(u'i love you .')

In [ ]:
# BeamSearchDecoder
def beam_evaluate_sentence(sentence, beam_width=3):
  sentence = preprocess_sentence(sentence)

  inputs = [en_tokenizer.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                          maxlen=max_length_en,
                                                          padding='post')
  inputs = tf.convert_to_tensor(inputs)
  inference_batch_size = inputs.shape[0]
  result = ''

  enc_start_state = [tf.zeros((inference_batch_size, units)), tf.zeros((inference_batch_size,units))]
  enc_out, enc_h, enc_c = encoder(inputs, enc_start_state)

  dec_h = enc_h
  dec_c = enc_c

  start_tokens = tf.fill([inference_batch_size], vi_tokenizer.word_index['<start>'])
  end_token = vi_tokenizer.word_index['<end>']

  enc_out = tfa.seq2seq.tile_batch(enc_out, multiplier=beam_width)
  decoder.attention_mechanism.setup_memory(enc_out)
  print("beam_with * [batch_size, max_length_en, rnn_units] :  3 * [1, 16, 1024]] :", enc_out.shape)

  hidden_state = tfa.seq2seq.tile_batch([enc_h, enc_c], multiplier=beam_width)
  decoder_initial_state = decoder.rnn_cell.get_initial_state(batch_size=beam_width*inference_batch_size, dtype=tf.float32)
  decoder_initial_state = decoder_initial_state.clone(cell_state=hidden_state)

  decoder_instance = tfa.seq2seq.BeamSearchDecoder(decoder.rnn_cell,beam_width=beam_width, output_layer=decoder.fc)
  decoder_embedding_matrix = decoder.embedding.variables[0]

  outputs, final_state, sequence_lengths = decoder_instance(decoder_embedding_matrix, start_tokens=start_tokens, end_token=end_token, initial_state=decoder_initial_state)

  final_outputs = tf.transpose(outputs.predicted_ids, perm=(0,2,1))
  beam_scores = tf.transpose(outputs.beam_search_decoder_output.scores, perm=(0,2,1))

  return final_outputs.numpy(), beam_scores.numpy()

def beam_translate(sentence):
  result, beam_scores = beam_evaluate_sentence(sentence)
  print(result.shape, beam_scores.shape)
  for beam, score in zip(result, beam_scores):
    print(beam.shape, score.shape)
    output = vi_tokenizer.sequences_to_texts(beam)
    output = [a[:a.index('<end>')] for a in output]
    beam_score = [a.sum() for a in score]
    print('Input: %s' % (sentence))
    for i in range(len(output)):
      print('{} Predicted translation: {}  {}'.format(i+1, output[i], beam_score[i]))

beam_translate(u'i love you .')

In [ ]:
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()

#BLEU score